<a href="https://colab.research.google.com/github/Michelleagl/A01/blob/main/Series_de_tiempo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, GRU


In [18]:
NatGas = yf.Ticker('NG=F')
NatGas


yfinance.Ticker object <NG=F>

In [19]:
META = yf.Ticker('META')
META


yfinance.Ticker object <META>

## **Natural Gas** future

In [20]:
futuro = 'NG=F'
ticker = yf.Ticker(futuro)
hist = ticker.history(start= '2024-01-01', end= '2024-06-30')

hist

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2024-01-02 00:00:00-05:00,2.605,2.675,2.524,2.568,132454,0.0,0.0
2024-01-03 00:00:00-05:00,2.575,2.719,2.537,2.668,166472,0.0,0.0
2024-01-04 00:00:00-05:00,2.688,2.863,2.681,2.821,206311,0.0,0.0
2024-01-05 00:00:00-05:00,2.845,2.906,2.681,2.893,187503,0.0,0.0
2024-01-08 00:00:00-05:00,2.950,3.012,2.694,2.980,237672,0.0,0.0
...,...,...,...,...,...,...,...
2024-06-24 00:00:00-04:00,2.670,2.834,2.635,2.811,45418,0.0,0.0
2024-06-25 00:00:00-04:00,2.816,2.844,2.699,2.756,78233,0.0,0.0
2024-06-26 00:00:00-04:00,2.737,2.746,2.613,2.628,141345,0.0,0.0


In [23]:
serie1 = MinMaxScaler(feature_range=(0,1))
scaled_data1 = serie1.fit_transform(hist['Close'].values.reshape(-1,1))

scaled_data1

array([[5.71346420e-01],
       [6.28883768e-01],
       [7.16916060e-01],
       [7.58342867e-01],
       [8.08400483e-01],
       [9.29229049e-01],
       [8.42347559e-01],
       [8.75719165e-01],
       [1.00000000e+00],
       [7.62370605e-01],
       [7.45109264e-01],
       [6.45569639e-01],
       [5.43153079e-01],
       [4.85615595e-01],
       [5.03452268e-01],
       [6.13348697e-01],
       [5.73072554e-01],
       [6.54200173e-01],
       [5.26467207e-01],
       [2.88837675e-01],
       [3.02071279e-01],
       [2.73302605e-01],
       [2.89988477e-01],
       [2.91714611e-01],
       [2.49712334e-01],
       [2.25546593e-01],
       [1.96777920e-01],
       [1.56501708e-01],
       [1.11047162e-01],
       [6.55926170e-02],
       [1.95626706e-02],
       [3.45219968e-03],
       [1.95626706e-02],
       [5.75332318e-04],
       [1.13924030e-01],
       [9.03336900e-02],
       [1.61104709e-02],
       [4.83314128e-02],
       [2.30149389e-02],
       [1.78365914e-01],


In [25]:
prediction_days = 60

x1 = []
y1 = []

for x in range(prediction_days, len(scaled_data1)):
    x1.append(scaled_data1[x-prediction_days:x, 0])
    y1.append(scaled_data1[x, 0])

x1, y1 = np.array(x1), np.array(y1)
x1 = np.reshape(x1, (x1.shape[0], x1.shape[1], 1))

x1.shape

(64, 60, 1)

In [27]:
model1 = Sequential()

model1.add(GRU(units=50, return_sequences=True, input_shape=(x1.shape[1], 1)))
model1.add(Dropout(0.2))
model1.add(GRU(units=50, return_sequences=True))
model1.add(Dropout(0.2))
model1.add(GRU(units=50))
model1.add(Dropout(0.2))
model1.add(Dense(units=1))

model1.compile(optimizer='adam', loss='mean_squared_error')
model1.fit(x1, y1, epochs=25, batch_size=32)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 103ms/step - loss: 0.2168
Epoch 2/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - loss: 0.1188
Epoch 3/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step - loss: 0.0529
Epoch 4/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - loss: 0.0160
Epoch 5/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - loss: 0.0328
Epoch 6/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - loss: 0.0369
Epoch 7/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - loss: 0.0220
Epoch 8/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step - loss: 0.0191
Epoch 9/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - loss: 0.0166
Epoch 10/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - loss: 0.0240
Epoch 11/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - loss: 0.0178
Epoch 12/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - loss: 0.0228
Epoch 13/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - loss: 0.0139
Epoch 14/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - loss: 0.0137
Epoch 15/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - loss: 0.0107
Epoch 16/25
2/2 ━━━━━━━━━━━━━━━

In [36]:
hist_test = ticker.history(start= '2024-01-01', end= '2024-06-30')
actual_price = hist_test['Close'].values

total_dataset = pd.concat((hist['Close'], hist_test['Close']), axis=0)
model_inputs = total_dataset[len(total_dataset) - len(hist_test) - prediction_days:].values
model_inputs = serie1.transform(model_inputs.reshape(-1, 1))

model_inputs

array([[1.14499431e-01],
       [1.20828498e-01],
       [1.54775574e-01],
       [1.70886045e-01],
       [1.78365914e-01],
       [1.08745696e-01],
       [1.12197896e-01],
       [6.67433502e-02],
       [9.03336900e-02],
       [7.88262206e-02],
       [1.04718027e-01],
       [1.01841159e-01],
       [1.24280766e-01],
       [1.36363636e-01],
       [4.48791445e-02],
       [3.62485425e-02],
       [2.24395380e-02],
       [2.61795136e-01],
       [2.39355598e-01],
       [2.05408522e-01],
       [2.64672072e-01],
       [3.26237019e-01],
       [3.56731827e-01],
       [3.63636364e-01],
       [3.52128894e-01],
       [4.17721580e-01],
       [3.89528238e-01],
       [4.63751458e-01],
       [4.42462653e-01],
       [4.83889460e-01],
       [5.29344006e-01],
       [6.04718027e-01],
       [6.76639779e-01],
       [6.30609902e-01],
       [7.28998861e-01],
       [6.22554700e-01],
       [5.43728411e-01],
       [5.84004554e-01],
       [5.28193341e-01],
       [5.73647887e-01],


In [42]:
x1_test = []

for x in range(prediction_days, len(model_inputs)):
    x1_test.append(model_inputs[x-prediction_days:x, 0])

  x1_test = np.array(x1_test)
  x1_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

  prediction_prices = model1.predict(x_test)
  prediction_prices = serie1.inverse_transform(prediction_prices)

prediction_prices

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 6)

# **META** stock

In [16]:
stock = 'META'
ticker = yf.Ticker(stock)
hist2 = ticker.history(start= '2024-01-01', end= '2024-06-30')

hist2

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2024-01-02 00:00:00-05:00,350.599548,352.435771,339.312744,345.579865,19042200,0.0,0.0
2024-01-03 00:00:00-05:00,344.272544,347.236455,342.476217,343.763580,15451100,0.0,0.0
2024-01-04 00:00:00-05:00,343.793520,347.436029,342.695770,346.408142,12099900,0.0,0.0
2024-01-05 00:00:00-05:00,346.278421,352.775081,345.549937,351.228271,13920700,0.0,0.0
2024-01-08 00:00:00-05:00,353.972629,358.243851,351.328039,357.924500,13890200,0.0,0.0
...,...,...,...,...,...,...,...
2024-06-24 00:00:00-04:00,499.200012,507.799988,494.290009,498.910004,13525300,0.0,0.0
2024-06-25 00:00:00-04:00,497.049988,510.709991,495.500000,510.600006,12109800,0.0,0.0
2024-06-26 00:00:00-04:00,506.649994,513.809998,504.679993,513.119995,8882300,0.0,0.0


In [24]:
serie2 = MinMaxScaler(feature_range=(0,1))
scaled_data2 = serie2.fit_transform(hist2['Close'].values.reshape(-1,1))

scaled_data2


array([[0.00992216],
       [0.        ],
       [0.01444694],
       [0.04077877],
       [0.07735951],
       [0.07065393],
       [0.14174383],
       [0.13738259],
       [0.16365956],
       [0.12533432],
       [0.13029524],
       [0.17260044],
       [0.21250679],
       [0.20340237],
       [0.22204717],
       [0.25203137],
       [0.26555155],
       [0.27078537],
       [0.30829271],
       [0.30305922],
       [0.24897867],
       [0.27427436],
       [0.71155394],
       [0.62661661],
       [0.60104833],
       [0.68211473],
       [0.68435003],
       [0.67404627],
       [0.67835316],
       [0.63048737],
       [0.70223145],
       [0.76083712],
       [0.70244968],
       [0.69389041],
       [0.67631741],
       [0.77509768],
       [0.76363692],
       [0.75113937],
       [0.78011844],
       [0.76358224],
       [0.79692756],
       [0.86334479],
       [0.84091474],
       [0.7974187 ],
       [0.82945398],
       [0.91731931],
       [0.88326479],
       [0.761

In [30]:
x2 = []
y2 = []

for x in range(prediction_days, len(scaled_data2)):
    x2.append(scaled_data1[x-prediction_days:x, 0])
    y2.append(scaled_data1[x, 0])

x2, y2 = np.array(x2), np.array(y2)
x2 = np.reshape(x2, (x2.shape[0], x2.shape[1], 1))

x2.shape

(64, 60, 1)

In [31]:
model2 = Sequential()

model2.add(GRU(units=50, return_sequences=True, input_shape=(x2.shape[1], 1)))
model2.add(Dropout(0.2))
model2.add(GRU(units=50, return_sequences=True))
model2.add(Dropout(0.2))
model2.add(GRU(units=50))
model2.add(Dropout(0.2))
model2.add(Dense(units=1))

model2.compile(optimizer='adam', loss='mean_squared_error')
model2.fit(x2, y2, epochs=25, batch_size=32)

Epoch 1/25


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 158ms/step - loss: 0.2246
Epoch 2/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 148ms/step - loss: 0.1315
Epoch 3/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - loss: 0.0569
Epoch 4/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - loss: 0.0209
Epoch 5/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - loss: 0.0290
Epoch 6/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step - loss: 0.0408
Epoch 7/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step - loss: 0.0283
Epoch 8/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - loss: 0.0174
Epoch 9/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - loss: 0.0166
Epoch 10/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - loss: 0.0219
Epoch 11/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - loss: 0.0261
Epoch 12/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - loss: 0.0166
Epoch 13/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - loss: 0.0151
Epoch 14/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step - loss: 0.0137
Epoch 15/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step - loss: 0.0107
Epoch 16/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 